In [57]:
#Studnet: Jhuma kabir meem
#Exercise Cuda basic
# !pip install pycuda

In [22]:
# -- initialize the device
import numpy as np
import pycuda.driver as cuda
import pycuda.autoinit
import pycuda.gpuarray as gpuarray
import numpy as np
from pycuda import driver, compiler, gpuarray, tools
import pycuda.autoinit

In [51]:
###### Part 1
# Generate two random vectors and implement CUDA kernel performing elemenwise add operation
N = 5 # length of array

# --- Create random vectors on the CPU
A = np.random.randn(1, N)
B = np.random.randn(1, N)

# --- Set CPU arrays as single precision
A = A.astype(np.float32)
B = B.astype(np.float32)

a = gpuarray.to_gpu(A)
b = gpuarray.to_gpu(B)
add_a_b = gpuarray.empty_like(a)


from pycuda.elementwise import ElementwiseKernel

# CUDA element wise add
lin_comb = ElementwiseKernel(
        "float *add_a_b, float *a, float *b",
        "add_a_b[i] =  a[i] + b[i]")

lin_comb(add_a_b, a, b,)

# Get results from device to host
h_c = add_a_b.get()


print("-" * 10)
print ("Random vectors A:")
print(a)

print("-" * 10)
print ("Random vectors B:")
print(b)

print("-" * 10)
print ("CUDA kernel performing elemenwise add operation of  Vector A and B:")
print(add_a_b)

print("-" * 10)
# Compare CPU and GPU addition result
if np.array_equal(h_c,  A +  B):
  print("CPU and GPU addition produced same result")
else :
  print("Error!: CPU and GPU doesnot produce same result")

# --- Flush context printf buffer
cuda.Context.synchronize()

----------
Random vectors A:
[[-0.34502926 -0.78622866  1.0008224  -1.8360682   0.6735068 ]]
----------
Random vectors B:
[[-1.1676087  -0.74887156 -1.1301663   1.1365993  -1.355186  ]]
----------
CUDA kernel performing elemenwise add operation of  Vector A and B:
[[-1.512638   -1.5351002  -0.12934387 -0.69946885 -0.6816792 ]]
----------
CPU and GPU addition produced same result


In [61]:
###### Part 2
#Generate two random matrices and implement CUDA kernel performing matrix multiplication


# kernel code for matrix multiplication

kernel_matrix_multiplication_code = """
__global__ void MatrixMulKernel(float *a, float *b, float *c)
{
    int tx = threadIdx.x;
    int ty = threadIdx.y;
    float P_value = 0;

    for (int k = 0; k < %(m_size)s; ++k) {
        float a_e = a[ty * %(m_size)s + k];
        float b_e = b[k * %(m_size)s + tx];
        P_value += a_e * b_e;
    }
    c[ty * %(m_size)s + tx] = P_value;
}
"""


m_size = 3 # define matrix size

A = np.random.randn(m_size, m_size).astype(np.float32) # create two random square matrices
B = np.random.randn(m_size, m_size).astype(np.float32)

multiplication_cpu = np.dot(A, B) # compute cpu matrix multiplication to compare  GPU computation


a_gpu = gpuarray.to_gpu(A) # transfer host (CPU) memory to device (GPU) memory 
b_gpu = gpuarray.to_gpu(B)

# create empty gpu array 
multiplication_gpu = gpuarray.empty((m_size, m_size), np.float32)

# get the kernel code  and  by specifying matrix size
kernel_code = kernel_matrix_multiplication_code % {
    'm_size': m_size 
    }

# compile the kernel code 
mod = compiler.SourceModule(kernel_code)

# get the kernel function from the compiled module
matrixmul = mod.get_function("MatrixMulKernel")

# call the kernel on the card
matrixmul( a_gpu, b_gpu, multiplication_gpu, block = (m_size, m_size, 1),
    )

#print the results
print("-" * 10)
print ("Random Matrix A:")
print (a_gpu.get())

print ("-" * 10)
print ("Random Matrix B:")
print (b_gpu.get())

print ("-" * 10)
print ("Matrix multiplication of A and B (GPU):")
print (multiplication_gpu.get())

print ("-" * 10)
if np.array_equal(multiplication_cpu,  multiplication_gpu.get()):
  print("CPU and GPU multuiplication  produced same result")
else :
  print("Error!: CPU and GPU does not produce same result")

----------
Random Matrix A:
[[ 1.6640799  -0.06716456 -1.5063071 ]
 [-0.25112775  0.11923535 -0.23239048]
 [-0.79191923 -0.47793746  1.6814418 ]]
----------
Random Matrix B:
[[-0.08376075 -1.3749142  -1.5821311 ]
 [ 1.0775951  -0.25780284  0.0922598 ]
 [-0.57076025 -1.181374    0.9831074 ]]
----------
Matrix multiplication of A and B (GPU):
[[ 0.64797944 -0.4911398  -4.1198506 ]
 [ 0.28216133  0.5890799   0.17985286]
 [-1.4083915  -0.7743769   2.8618634 ]]
----------
CPU and GPU multuiplication  produced same result
